In this project, we aim to investigate the sources of spatial heterogeneity in glioblastoma multiforme using mathematical modeling.
We include the following sources of heterogeneity:
- genetic heterogeneity
- phenotypic heterogeneity (go-or-grow)

To this end, we adapt a CA model of glioblastoma multiforme from [1] that includes the go-or-grow hypothesis.
We then investigate the effects of genetic heterogeneity on the growth dynamics of the tumor.

### Plan
Things to consider:
- Should we include oxygen? If so, include a hypoxic region in the center of the tumor?
- Should we include directed motion, e.g., based on an oxygen gradient?
Let's start with this:
1. Change go-or-grow model to include a realistic mutation probability
2. A mutation might change: Proliferation rate, or phenotypic switch parameter
 

In [1]:
from lgca import get_lgca
from lgca.nove_ib_interactions import go_or_grow_glioblastoma

In [2]:
lgca = get_lgca(geometry='hex', dims=(50, 50), interaction='go_or_grow_kappa')

interaction go_or_grow_kappa is not defined! Random walk used instead.
Implemented interactions: ['go_and_grow', 'go_or_grow', 'alignment', 'aggregation', 'random_walk', 'excitable_medium', 'nematic', 'persistant_motion', 'chemotaxis', 'contact_guidance', 'only_propagation']
